In [1]:
import glob

bench_dir = "/data/PycharmProjects/cytof_benchmark/results/latent_data/"
latent_files = glob.glob(bench_dir + "*_latent.csv")
latent_files

['/data/PycharmProjects/cytof_benchmark/results/latent_data/CafDataset_WAE_MMD_val_latent.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/CafDataset_BetaVAE_train_latent.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/CafDataset_BetaVAE_val_latent.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/ChallengeDataset_BetaVAE_train_latent.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/CafDataset_HyperSphericalVAE_val_latent.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/CafDataset_WAE_MMD_train_latent.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/ChallengeDataset_WAE_MMD_val_latent.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/OrganoidDataset_BetaVAE_train_latent.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/ChallengeDataset_WAE_MMD_train_latent.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/OrganoidDataset_DBetaVAE_train_la

In [2]:
import pandas as pd

latent_dict = dict()
for latent_file in latent_files:
    filename = latent_file.split('/')[-1]
    dataset_name = filename.split('_')[0]
    split = filename.split('_')[-2]
    model_name = filename.split('_',maxsplit=1)[1].removesuffix('_latent.csv').removesuffix('_val').removesuffix('_train')
    print(dataset_name,model_name,split)
    if dataset_name not in latent_dict:
        latent_dict[dataset_name]=dict()
    if model_name not in latent_dict[dataset_name]:
        latent_dict[dataset_name][model_name]=dict()
    latent_dict[dataset_name][model_name][split]= pd.read_csv(latent_file,index_col=0)

CafDataset WAE_MMD val
CafDataset BetaVAE train
CafDataset BetaVAE val
ChallengeDataset BetaVAE train
CafDataset HyperSphericalVAE val
CafDataset WAE_MMD train
ChallengeDataset WAE_MMD val
OrganoidDataset BetaVAE train
ChallengeDataset WAE_MMD train
OrganoidDataset DBetaVAE train
ChallengeDataset HyperSphericalVAE train
OrganoidDataset HyperSphericalVAE val
CafDataset DBetaVAE val
OrganoidDataset WAE_MMD val
ChallengeDataset DBetaVAE val
OrganoidDataset WAE_MMD train
ChallengeDataset HyperSphericalVAE val
OrganoidDataset HyperSphericalVAE train
OrganoidDataset DBetaVAE val
ChallengeDataset BetaVAE val
ChallengeDataset DBetaVAE train
CafDataset DBetaVAE train
CafDataset HyperSphericalVAE train
OrganoidDataset BetaVAE val


In [3]:
latent_dict['OrganoidDataset']['BetaVAE']

{'train':             VAE1      VAE2
 0       0.730659 -1.110678
 1       0.582513  1.452487
 2       1.624205 -0.158508
 3      -0.222219  1.461618
 4      -0.458737 -0.741948
 ...          ...       ...
 820728 -1.659552 -0.749211
 820729 -1.074803  0.265936
 820730 -1.149795  0.754065
 820731  0.752049  0.019883
 820732 -1.121911 -1.272193
 
 [820733 rows x 2 columns],
 'val':             VAE1      VAE2
 0       0.034673 -1.756849
 1      -0.752730 -0.288588
 2      -1.204419  1.593389
 3      -0.374708  0.467037
 4       0.541722  0.093536
 ...          ...       ...
 234490  1.599098  1.048390
 234491  0.090505 -1.004703
 234492 -0.056494  0.331654
 234493 -0.448628 -0.740973
 234494 -1.251991 -0.041530
 
 [234495 rows x 2 columns]}

In [4]:
import sys
if ".." not in sys.path:
    sys.path.insert(0, "..")

In [5]:
from datasets import OrganoidDataset
data = OrganoidDataset(data_dir='../data/organoids/')
X_train,y_train = data.train
X_val,y_val = data.val

In [6]:
y_train

,index,cell_type,day
0,156737,Enterocyte,2
1,1151315,Tuft,7
2,611947,Enteroendocrine,3
3,756795,Goblet,6
4,213350,Enterocyte,2
...,...,...,...
820728,2685,Enterocyte,1
820729,1122479,Tuft,5
820730,253852,Enterocyte,3
820731,746001,Goblet,5


In [7]:
y_train['cell_type'].value_counts(normalize=True)

Enterocyte         0.505006
Stem               0.205210
Tuft               0.110819
Enteroendocrine    0.092274
Goblet             0.064896
Paneth             0.021795
Name: cell_type, dtype: float64

In [8]:
y_train['day'].value_counts(normalize=True)

2    0.165896
3    0.158224
5    0.143529
6    0.142414
7    0.138300
4    0.136753
1    0.114884
Name: day, dtype: float64

In [9]:
import xgboost as xgb
import sklearn
from sklearn.decomposition import PCA

for var in ['cell_type','day']:
    n_classes = len(y_train[var].astype('category').cat.categories)
    reg = xgb.XGBRegressor(tree_method="gpu_hist", objective='multi:softmax',num_class=n_classes)
    reg.fit(X_train, y_train[var].astype('category').cat.codes)
    y_pred = reg.predict(X_val)
    print(f"Accuracy score for {var}({n_classes}) prediction for OrganoidDataset on all biomarkers:",
          sklearn.metrics.accuracy_score(y_val[var].astype('category').cat.codes, y_pred))

    pca = PCA(n_components=2)
    pca.fit(X_train)

    reg = xgb.XGBRegressor(tree_method="gpu_hist", objective='multi:softmax',num_class=n_classes)
    reg.fit(pca.transform(X_train), y_train[var].astype('category').cat.codes)
    y_pred = reg.predict(pca.transform(X_val))
    print(f"Accuracy score for {var}({n_classes}) prediction for OrganoidDataset on 2 principal components:",
          sklearn.metrics.accuracy_score(y_val[var].astype('category').cat.codes, y_pred))

    for model_name in latent_dict['OrganoidDataset'].keys():
        reg = xgb.XGBRegressor(tree_method="gpu_hist", objective='multi:softmax',num_class=n_classes)
        reg.fit(latent_dict['OrganoidDataset'][model_name]['train'], y_train[var].astype('category').cat.codes)
        y_pred = reg.predict(latent_dict['OrganoidDataset'][model_name]['val'])
        print(f"Accuracy score for {var}({n_classes}) prediction for OrganoidDataset on 2 AE components with {model_name}:",
              sklearn.metrics.accuracy_score(y_val[var].astype('category').cat.codes, y_pred))

Accuracy score for cell_type(6) prediction for OrganoidDataset on all biomarkers: 0.8730889784430371
Accuracy score for cell_type(6) prediction for OrganoidDataset on 2 principal components: 0.7289451800678053
Accuracy score for cell_type(6) prediction for OrganoidDataset on 2 AE components with BetaVAE: 0.8047079895093712
Accuracy score for cell_type(6) prediction for OrganoidDataset on 2 AE components with DBetaVAE: 0.8056077954753833
Accuracy score for cell_type(6) prediction for OrganoidDataset on 2 AE components with HyperSphericalVAE: 0.8024094330369518
Accuracy score for cell_type(6) prediction for OrganoidDataset on 2 AE components with WAE_MMD: 0.7963154864709269
Accuracy score for day(7) prediction for OrganoidDataset on all biomarkers: 0.4340263118616602
Accuracy score for day(7) prediction for OrganoidDataset on 2 principal components: 0.2954348706795454
Accuracy score for day(7) prediction for OrganoidDataset on 2 AE components with BetaVAE: 0.3201646090534979
Accuracy sco

In [10]:
from datasets import CafDataset
data = CafDataset(data_dir='../data/caf/')
X_train,y_train = data.train
X_val,y_val = data.val
y_val

,index,Date,Patient,Culture,Treatment,Concentration,Replicate,Cell_type,pPKCa,Plate,Batch
0,2840618,20210518,5,PDO,S,2,C,PDOs,48.466751,SLV,1
1,6904274,20210330,27,PDOF,VS,3,A,PDOs,NaN,SLV,1
2,5478019,20210420,75,PDO,S,1,A,PDOs,NaN,SLV,1
3,7230165,20210525,216,PDOF,VS,2,C,Fibs,8.220644,SLV,1
4,7643022,20210524,109,PDOF,VS,2,A,Fibs,27.408182,SLV,1
...,...,...,...,...,...,...,...,...,...,...,...
4951201,5232280,20210608,11,F,S,1,C,Fibs,56.341122,SLV,1
4951202,17554307,20210524,109,PDOF,F,3,B,PDOs,70.352081,CSF,1
4951203,8748653,20210518,5,PDOF,L,1,A,Fibs,4.410349,SLV,1
4951204,17634951,20210524,109,F,F,4,B,Fibs,24.214802,CSF,1


In [11]:
y_train['Patient'].value_counts(normalize=True)

109    0.156703
216    0.125141
11     0.119648
23     0.115158
5      0.107257
141    0.091459
27     0.082281
99     0.068535
75     0.067918
21     0.065899
Name: Patient, dtype: float64

In [12]:
y_train['Culture'].value_counts(normalize=True)

PDOF    0.429550
F       0.350909
PDO     0.219541
Name: Culture, dtype: float64

In [13]:
y_train['Treatment'].value_counts(normalize=True)

F       0.147379
L       0.146690
S       0.134319
VS      0.105978
O       0.101107
DMSO    0.076687
V       0.040261
C       0.039759
AH      0.038987
CS      0.035903
CF      0.034394
CSF     0.034107
SF      0.033300
H2O     0.031130
Name: Treatment, dtype: float64

In [14]:
for var in ['Patient','Culture','Treatment']:
    n_classes = len(y_train[var].astype('category').cat.categories)
    reg = xgb.XGBRegressor(tree_method="gpu_hist", objective='multi:softmax',num_class=n_classes)
    reg.fit(X_train, y_train[var].astype('category').cat.codes)
    y_pred = reg.predict(X_val)
    print(f"Accuracy score for {var}({n_classes}) prediction for CafDataset on all biomarkers:",
          sklearn.metrics.accuracy_score(y_val[var].astype('category').cat.codes, y_pred))

    pca = PCA(n_components=2)
    pca.fit(X_train)

    reg = xgb.XGBRegressor(tree_method="gpu_hist", objective='multi:softmax',num_class=n_classes)
    reg.fit(pca.transform(X_train), y_train[var].astype('category').cat.codes)
    y_pred = reg.predict(pca.transform(X_val))
    print(f"Accuracy score for {var}({n_classes}) prediction for CafDataset on 2 principal components:",
          sklearn.metrics.accuracy_score(y_val[var].astype('category').cat.codes, y_pred))

    for model_name in latent_dict['CafDataset'].keys():
        reg = xgb.XGBRegressor(tree_method="gpu_hist", objective='multi:softmax',num_class=n_classes)
        reg.fit(latent_dict['CafDataset'][model_name]['train'], y_train[var].astype('category').cat.codes)
        y_pred = reg.predict(latent_dict['CafDataset'][model_name]['val'])
        print(f"Accuracy score for {var}({n_classes}) prediction for CafDataset on 2 AE components with {model_name}:",
              sklearn.metrics.accuracy_score(y_val[var].astype('category').cat.codes, y_pred))

Accuracy score for Patient(10) prediction for CafDataset on all biomarkers: 0.8986951865868639
Accuracy score for Patient(10) prediction for CafDataset on 2 principal components: 0.265859469389882
Accuracy score for Patient(10) prediction for CafDataset on 2 AE components with WAE_MMD: 0.6426060640579285
Accuracy score for Patient(10) prediction for CafDataset on 2 AE components with BetaVAE: 0.6842409707857036
Accuracy score for Patient(10) prediction for CafDataset on 2 AE components with HyperSphericalVAE: 0.6989406217394307
Accuracy score for Patient(10) prediction for CafDataset on 2 AE components with DBetaVAE: 0.662182304674861
Accuracy score for Culture(3) prediction for CafDataset on all biomarkers: 0.6988822521220083
Accuracy score for Culture(3) prediction for CafDataset on 2 principal components: 0.5726182671454187
Accuracy score for Culture(3) prediction for CafDataset on 2 AE components with WAE_MMD: 0.6076176188185263
Accuracy score for Culture(3) prediction for CafDatas

In [15]:
from datasets import ChallengeDataset
data = ChallengeDataset(data_dir='../data/breast_cancer_challenge/')
X_train,y_train = data.train
X_val,y_val = data.val
y_val

,index,treatment,cell_line,time,cellID,fileID
0,855587,full,MCF10F,0.0,28353,2000
1,364135,iPKC,HBL100,60.0,3115,1531
2,356676,iPI3K,OCUBM,0.0,4744,2464
3,248118,iMEK,HBL100,40.0,4960,1513
4,142322,iMEK,EVSAT,0.0,898,1892
...,...,...,...,...,...,...
2204463,224723,iMEK,184A1,13.0,2845,2676
2204464,121261,EGF,CAL851,60.0,36,1451
2204465,350172,iPKC,CAL51,0.0,6718,1217
2204466,350589,iPKC,MCF10F,17.0,2249,395


In [16]:
y_train['treatment'].value_counts(normalize=True)

EGF      0.286634
iPI3K    0.188677
iEGFR    0.177923
iPKC     0.155368
iMEK     0.149207
full     0.042192
Name: treatment, dtype: float64

In [17]:
y_train['cell_line'].value_counts(normalize=True)

MCF10F         0.082939
MCF10A         0.051125
HCC2185        0.050523
HCC2157        0.048433
MX1            0.044457
MPE600         0.042415
ZR7530         0.041559
CAL851         0.040501
OCUBM          0.040390
HCC3153        0.038702
BT474          0.038168
UACC812        0.037531
BT20           0.037402
HDQP1          0.037086
CAL51          0.036507
JIMT1          0.034318
EFM192A        0.033857
184A1          0.033757
HBL100         0.033320
MCF7           0.033261
MFM223         0.032877
MDAMB415       0.031609
EVSAT          0.025269
CAL148         0.022048
MDAMB157       0.015586
MDAMB175VII    0.014699
DU4475         0.014057
HCC1599        0.007604
Name: cell_line, dtype: float64

In [29]:
for var in ['treatment','cell_line']:
    n_classes = len(y_train[var].astype('category').cat.categories)
    reg = xgb.XGBRegressor(tree_method="gpu_hist", objective='multi:softmax',num_class=n_classes)
    reg.fit(X_train, y_train[var].astype('category').cat.codes)
    y_pred = reg.predict(X_val)
    print(f"Accuracy score for {var}({n_classes}) prediction for ChallengeDataset on all biomarkers:",
          sklearn.metrics.accuracy_score(y_val[var].astype('category').cat.codes, y_pred))

    pca = PCA(n_components=2)
    pca.fit(X_train)

    reg = xgb.XGBRegressor(tree_method="gpu_hist", objective='multi:softmax',num_class=n_classes)
    reg.fit(pca.transform(X_train), y_train[var].astype('category').cat.codes)
    y_pred = reg.predict(pca.transform(X_val))
    print(f"Accuracy score for {var}({n_classes}) prediction for ChallengeDataset on 2 principal components:",
          sklearn.metrics.accuracy_score(y_val[var].astype('category').cat.codes, y_pred))

    for model_name in latent_dict['ChallengeDataset'].keys():
        reg = xgb.XGBRegressor(tree_method="gpu_hist", objective='multi:softmax',num_class=n_classes)
        reg.fit(latent_dict['ChallengeDataset'][model_name]['train'], y_train[var].astype('category').cat.codes)
        y_pred = reg.predict(latent_dict['ChallengeDataset'][model_name]['val'])
        print(f"Accuracy score for {var}({n_classes}) prediction for ChallengeDataset on 2 AE components with {model_name}:",
              sklearn.metrics.accuracy_score(y_val[var].astype('category').cat.codes, y_pred))

    reg = xgb.XGBRegressor(tree_method="gpu_hist", objective='multi:softmax',num_class=n_classes)


Accuracy score for treatment(6) prediction for ChallengeDataset on all biomarkers: 0.4927828392156294
Accuracy score for treatment(6) prediction for ChallengeDataset on 2 principal components: 0.28790256878303516
Accuracy score for treatment(6) prediction for ChallengeDataset on 2 AE components with BetaVAE: 0.3215764529129023
Accuracy score for treatment(6) prediction for ChallengeDataset on 2 AE components with WAE_MMD: 0.33624212281602633
Accuracy score for treatment(6) prediction for ChallengeDataset on 2 AE components with HyperSphericalVAE: 0.3266311872070722
Accuracy score for treatment(6) prediction for ChallengeDataset on 2 AE components with DBetaVAE: 0.3136266890696531
Accuracy score for cell_line(28) prediction for ChallengeDataset on all biomarkers: 0.855706682972944
Accuracy score for cell_line(28) prediction for ChallengeDataset on 2 principal components: 0.15130725417651786
Accuracy score for cell_line(28) prediction for ChallengeDataset on 2 AE components with BetaVAE: